Load files

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

datewise = pd.read_csv('datewise.csv')
datewise = pd.DataFrame(datewise)

In [2]:
datewise.head()

,ObservationDate,Confirmed,Recovered,Deaths,Days Since,WeekOfYear,Mortality Rate,Recovery Rate,Active Cases,Closed Cases
0,2020-01-22,555.0,28.0,17.0,0 days 00:00:00.000000000,4,3.063063,5.045045,510.0,45.0
1,2020-01-23,653.0,30.0,18.0,1 days 00:00:00.000000000,4,2.756508,4.594181,605.0,48.0
2,2020-01-24,941.0,36.0,26.0,2 days 00:00:00.000000000,4,2.763018,3.825717,879.0,62.0
3,2020-01-25,1438.0,39.0,42.0,3 days 00:00:00.000000000,4,2.920723,2.712100,1357.0,81.0
4,2020-01-26,2118.0,52.0,56.0,4 days 00:00:00.000000000,4,2.644004,2.455146,2010.0,108.0


In [3]:
datewise.index = datewise['ObservationDate']
datewise.index = pd.to_datetime(datewise.index)
datewise.drop(columns='ObservationDate')

,Confirmed,Recovered,Deaths,Days Since,WeekOfYear,Mortality Rate,Recovery Rate,Active Cases,Closed Cases
ObservationDate,,,,,,,,,
2020-01-22,555.0,28.0,17.0,0 days 00:00:00.000000000,4,3.063063,5.045045,510.0,45.0
2020-01-23,653.0,30.0,18.0,1 days 00:00:00.000000000,4,2.756508,4.594181,605.0,48.0
2020-01-24,941.0,36.0,26.0,2 days 00:00:00.000000000,4,2.763018,3.825717,879.0,62.0
2020-01-25,1438.0,39.0,42.0,3 days 00:00:00.000000000,4,2.920723,2.712100,1357.0,81.0
2020-01-26,2118.0,52.0,56.0,4 days 00:00:00.000000000,4,2.644004,2.455146,2010.0,108.0
...,...,...,...,...,...,...,...,...,...
2020-06-25,9609829.0,4838921.0,489312.0,155 days 00:00:00.000000000,26,5.091787,50.353872,4281596.0,5328233.0
2020-06-26,9801572.0,4945557.0,494181.0,156 days 00:00:00.000000000,26,5.041855,50.456774,4361834.0,5439738.0
2020-06-27,9979535.0,5051864.0,498710.0,157 days 00:00:00.000000000,26,4.997327,50.622238,4428961.0,5550574.0


## Prediction using Machine Learning Models

#### Linear Regression Model for Confirm Cases Prediction

In [235]:
datewise['Days Since'] = datewise.index - datewise.index[0]
datewise['Days Since'] = datewise['Days Since'].dt.days

In [383]:
train_ml = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid_ml = datewise.iloc[int(datewise.shape[0]*0.9):]
model_scores=[]

In [384]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(normalize=True)

In [385]:
import numpy as np

lin_reg.fit(np.array(train_ml['Days Since']).reshape(-1, 1),
           np.array(train_ml['Confirmed']).reshape(-1,1))

LinearRegression(normalize=True)

In [386]:
prediction_valid_linreg = lin_reg.predict(np.array(valid_ml['Days Since']).reshape(-1,1))

In [387]:
from sklearn.metrics import mean_squared_error

model_scores.append(np.sqrt(mean_squared_error(valid_ml['Confirmed'],
                                              prediction_valid_linreg)))
print('Root Mean Square Error for Linear Regression: ',
      np.sqrt( mean_squared_error(valid_ml['Confirmed'], prediction_valid_linreg) ) )

Root Mean Square Error for Linear Regression:  2665388.3159043086


In [388]:
import matplotlib.pyplot as plt

plt.figure(figsize=(11, 6))
prediction_linreg = lin_reg.predict(np.array(datewise['Days Since']).reshape(-1, 1))

linreg_output = []
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i][0])

<Figure size 792x432 with 0 Axes>

In [389]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=datewise.index, y=datewise['Confirmed'],
                        mode='lines+markers', name='Train Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=linreg_output,
                        mode='lines', name='Linear Regression Best Fit Line',
             line = dict(color='black', dash='dot')))

fig.update_layout(title='Confirmed Cases Linear Regression Prediction',
                 xaxis_title='Date', yaxis_title='Confirmed Cases', 
                  legend=dict(x=0, y=1))

fig.show()

The Linear Regression Model is absolutely falling apart. As it is clearly visible that the trend of Confirmed Cases in absolutely not Linear.

In [390]:
from datetime import timedelta

new_date = []
new_prediction_lr = []

for i in range(1, 100):
    new_date.append(datewise.index[-1]+timedelta(days=i))
    new_prediction_lr.append(lin_reg.predict(np.array(datewise['Days Since'].max() + i).reshape(-1,1))[0][0])

    

#### Polynomial Regression for Prediction of Confirmed Cases

In [391]:
train_ml = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid_ml = datewise.iloc[int(datewise.shape[0]*0.9):]

In [392]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree = 8)

In [393]:
train_poly=poly.fit_transform(np.array(train_ml["Days Since"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_ml["Days Since"]).reshape(-1,1))
y=train_ml["Confirmed"]

In [394]:
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y)

LinearRegression(normalize=True)

In [395]:
prediction_poly=linreg.predict(valid_poly)
rmse_poly=np.sqrt(mean_squared_error(valid_ml["Confirmed"],prediction_poly))
model_scores.append(rmse_poly)
print("Root Mean Squared Error for Polynomial Regression: ",rmse_poly)

Root Mean Squared Error for Polynomial Regression:  1983232.8167772032


In [396]:
comp_data=poly.fit_transform(np.array(datewise["Days Since"]).reshape(-1,1))
plt.figure(figsize=(11,6))
predictions_poly=linreg.predict(comp_data)

fig=go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=datewise.index, y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

<Figure size 792x432 with 0 Axes>

In [397]:
new_prediction_poly = []

for i in range(1, 100):
    new_date_poly = poly.fit_transform(np.array(datewise['Days Since'].max()+i).reshape(-1,1))
    new_prediction_poly.append(linreg.predict(new_date_poly)[0])

In [398]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

model_predictions = pd.DataFrame(zip(new_date, new_prediction_lr, new_prediction_poly),
                                columns=['Dates', 'Linear Regression Prediction', 'Polynomial Regression Prediction'])
model_predictions.head()

,Dates,Linear Regression Prediction,Polynomial Regression Prediction
0,2020-06-30,6899263.797007,5553059.654346
1,2020-07-01,6952906.075865,4970776.973441
2,2020-07-02,7006548.354724,4300888.528809
3,2020-07-03,7060190.633582,3535363.573804
4,2020-07-04,7113832.912440,2665639.382201


#### Support Vector Machine ModelRegressor for Prediction of Confirmed Cases

## Takes too long times ,so skip it

In [399]:
train_ml = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid_ml = datewise.iloc[int(datewise.shape[0]*0.9):]

In [400]:
from sklearn.svm import SVR

# Initializing SVR Model
svm = SVR(C=1, degree=5, kernel='poly', epsilon=0.01)

In [401]:
# Fitting model on the training data
svm.fit(np.array(train_ml['Days Since']).reshape(-1, 1), np.array(train_ml['Confirmed']).reshape(-1,1))

SVR(C=1, degree=5, epsilon=0.01, kernel='poly')

In [402]:
prediction_valid_svm = svm.predict(np.array(valid_ml['Days Since']).reshape(-1,1))

In [403]:
model_scores.append(np.sqrt(mean_square_error(valid_ml['Confirmed'], prediction_valid_svm)))
print('Root Mean Square Error for Support Vector Machine: ', np.sqrt(mean_squared_error(
    valid_ml['Confirmed'], prediction_valid_svm)))

NameError: name 'mean_square_error' is not defined

In [ ]:
plt.figure(figsize=(11, 6))
prediction_svm = svm.predict(np.array(datewise['Days Since']).reshape(-1,1))

fig = go.Figure()
fig.add_trace(go.Scatter(x=datewise.index, y=datewise['Confirmed'],
                        mode='lines+markers', name='Train Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=datewise.index, y=prediction_svm,
                        mode='lines', name='Support Vector Machine Best fit Kernel',
                        line=dict(color='black', dash='dot')))
fig.update_layout(title='Confirmed Cases Support Vector Machine Regressor Prediction',
                 xais_title='Date', yaxis_title='Confirmed Cases', 
                  legend = dict(x=0,y=1, 'normal'))

fig.show()

In [372]:
new_date = []
new_prediction_svm = []

for i in range(1, 100):
    new_date.append(datewise.index[-1]+timedelta(days=i))
    new_prediction_svm.append(svm.predict(np.array(datewise['Days Since'].max() + i).reshape(-1,1))[0])


NameError: name 'svm' is not defined

In [35]:
pd.set_option('display.float_format', lambda x: '%.6f' % x)

model_predictions = pd.DataFrame(zip(new_date_poly, new_prediction_lr, new_prediction_poly),
                                columns=['Dates', 'Linear Regression Prediction', 'Polynomial Regression Prediction', 'SVM Prediction'])
model_predictions.head()

NameError: name 'new_prediction_lr' is not defined

Predictions of Linear Regression are nowhere close to actual values.

## Time Series Forecasting

#### Holt's Linear Model

In [404]:
model_train = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid = datewise.iloc[int(datewise.shape[0]*0.9):]
y_pred = valid.copy()

In [405]:
from statsmodels.tsa.api import Holt, SimpleExpSmoothing, ExponentialSmoothing


holt = Holt(np.asarray(model_train['Confirmed'])).fit(
    smoothing_level=0.2, smoothing_slope=1.8, optimized=False)

In [406]:
y_pred['Holt'] = holt.forecast(len(valid))
model_scores.append(np.sqrt(mean_squared_error(y_pred['Confirmed'], y_pred['Holt'])))
print("Root Mean Square Error Holt's Linear Model: ", np.sqrt(mean_squared_error(y_pred['Confirmed'], y_pred['Holt'])))

Root Mean Square Error Holt's Linear Model:  182429.61166897498


In [407]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train['Confirmed'],
                        mode='lines+markers', name='Train Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Confirmed'],
                        mode='lines+markers', name='Validation Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred['Holt'],
                        mode='lines+markers', name='Prediction of Confirmed Cases'))
fig.update_layout(title="Confirmed Cases Holt's Linear Model Prediction", 
                 xaxis_title='Date', yaxis_title='Confirmed Cases', 
                  legend = dict(x=0,y=1,traceorder='normal'))

fig.show()

In [408]:
from datetime import timedelta

holt_new_date = []
holt_new_prediction = []

for i in range(1, 100):
    holt_new_date.append(datewise.index[-1]+timedelta(days=i))
    holt_new_prediction.append(holt.forecast((len(valid)+i))[-1])
    
model_predictions["Holt's Linear Model Prediction"] = holt_new_prediction
model_predictions.head()

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085


#### Holt's Winter Model for Daily Time Series

In [409]:
model_train = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid = datewise.iloc[int(datewise.shape[0]*0.9):]
y_pred = valid.copy()

In [410]:
import warnings
warnings.filterwarnings('ignore')
es = ExponentialSmoothing(np.asarray(model_train['Confirmed']),
                         seasonal_periods = 11, trend='mul', seasonal='add').fit()

In [411]:
y_pred["Holt's Winter Model"] = es.forecast(len(valid))
model_scores.append(np.sqrt(mean_squared_error(y_pred['Confirmed'],
                                              y_pred["Holt's Winter Model"])))
print("Root Mean Square Error for Holt's Winter Model: ",
     np.sqrt(mean_squared_error(y_pred['Confirmed'], y_pred["Holt's Winter Model"])))

Root Mean Square Error for Holt's Winter Model:  52727.7197348411


In [412]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train['Confirmed'],
                        mode='lines+markers', name='Train Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Confirmed'],
                        mode='lines+markers', name='Validation Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["Holt's Winter Model"],
                        mode='lines+markers', name='Prediction of Confirmed Cases'))
fig.update_layout(title="Confirmed Cases Holt's Winter Model Prediction",
                 xaxis_title='Date', yaxis_title='Confirmed Cases', 
                 legend= dict(x=0,y=1, traceorder='normal'))

fig.show()

In [413]:
holt_winter_new_prediction = []

for i in range(1, 100):
    holt_winter_new_prediction.append(es.forecast((len(valid)+i))[-1])
    
model_predictions["Holt's Winter Model Prediction"] = holt_winter_new_prediction
model_predictions.head()

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516,10573460.804313
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159,10761314.342326
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801,10953361.048453
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443,11147808.718753
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085,11345481.962612


#### AR Model (using AUTO ARIMA)

In [414]:
model_train = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid = datewise.iloc[int(datewise.shape[0]*0.9):]
y_pred = valid.copy()

In [330]:
from tqdm.notebook import tqdm   # 진행표시바
import time

for i in tqdm(range(100), desc="ing"):
    time.sleep(0.1)

In [331]:
from sys import executable
print(executable)

C:\Users\kheed\Anaconda3\python.exe


In [332]:
!cd

C:\Users\kheed\Desktop\PNU\Senior\학부연구생\covid19


In [333]:
!pip install pmdarima
# !를 사용함으로써 커맨드창에서 입력하는 것과 동일하게 처리가 가능하다.

In [123]:
!pip list

Package                            Version
---------------------------------- -----------
-tatsmodels                        0.10.1
-umpy                              1.16.5
absl-py                            0.9.0
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 1.9.7
anaconda-project                   0.8.3
appdirs                            1.4.4
asn1crypto                         1.0.1
astroid                            2.3.1
astropy                            3.2.1
astunparse                         1.6.3
atomicwrites                       1.3.0
attrs                              19.2.0
Babel                              2.7.0
backcall                           0.1.0
backports.functools-lru-cache      1.5
backports.os                       0.1.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0
backports.weakref                  1.0.post1
beautifulsoup4                     4.8.0
bita

scipy                              1.4.1
seaborn                            0.9.0
Send2Trash                         1.5.0
setuptools                         41.4.0
simplegeneric                      0.8.1
singledispatch                     3.4.0.3
six                                1.12.0
snowballstemmer                    2.0.0
sortedcollections                  1.1.2
sortedcontainers                   2.1.0
soupsieve                          1.9.3
Sphinx                             2.2.0
sphinxcontrib-applehelp            1.0.1
sphinxcontrib-devhelp              1.0.1
sphinxcontrib-htmlhelp             1.0.2
sphinxcontrib-jsmath               1.0.1
sphinxcontrib-qthelp               1.0.2
sphinxcontrib-serializinghtml      1.1.3
sphinxcontrib-websupport           1.1.2
spyder                             3.3.6
spyder-kernels                     0.5.2
SQLAlchemy                         1.3.9
statsmodels                        0.11.1
sympy                              1.4
tables       

In [124]:
!conda list

# packages in environment at C:\Users\kheed\Anaconda3:
#
# Name                    Version                   Build  Channel
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
absl-py                   0.9.0                    pypi_0    pypi
alabaster                 0.7.12                   py37_0  
anaconda                  2019.10                  py37_0  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.7                    py37_0  
anaconda-project          0.8.3                      py_0  
appdirs                   1.4.4                    pypi_0    pypi
asn1crypto                1.0.1                    py37_0  
astroid                   2.3.1                    py37_0  
astropy                   3.2.1            py37he774522_0  
astunparse                1.6.3                    pypi_0    pypi
atomicwrites              1.3.0                    py37_1  
attrs                     19.2.0                     py_0  
babel             

m2w64-gmp                 6.1.0                         2  
m2w64-libwinpthread-git   5.0.0.4634.697f757               2  
markdown                  3.2.2                    pypi_0    pypi
markupsafe                1.1.1            py37he774522_0  
matplotlib                3.1.1            py37hc8f65d3_0  
mccabe                    0.6.1                    py37_1  
menuinst                  1.4.16           py37he774522_0  
mistune                   0.8.4            py37he774522_0  
mkl                       2019.4                      245  
mkl-service               2.3.0            py37hb782905_0  
mkl_fft                   1.0.14           py37h14836fe_0  
mkl_random                1.1.0            py37h675688f_0  
mock                      3.0.5                    py37_0  
more-itertools            7.2.0                    py37_0  
mpmath                    1.1.0                    py37_0  
msgpack-python            0.6.1            py37h74a9793_1  
msys2-conda-epoch         20160

statsmodels               0.11.1                   pypi_0    pypi
sympy                     1.4                      py37_0  
tbb                       2019.4               h74a9793_0  
tblib                     1.4.0                      py_0  
tensorboard               2.2.2                    pypi_0    pypi
tensorboard-plugin-wit    1.6.0.post3              pypi_0    pypi
tensorflow                2.2.0                    pypi_0    pypi
tensorflow-estimator      2.2.0                    pypi_0    pypi
termcolor                 1.1.0                    pypi_0    pypi
terminado                 0.8.2                    py37_0  
testpath                  0.4.2                    py37_0  
tk                        8.6.8                hfa6e2cd_0  
toolz                     0.10.0                     py_0  
tornado                   6.0.3            py37he774522_0  
tqdm                      4.36.1                     py_0  
traitlets                 4.3.3                    py37_0  
tran

In [415]:
from pmdarima import *   # pyramid-arima 는 예전에 쓰던 모듈명
# from pmdarima import model_selection

In [416]:
model_ar = auto_arima(model_train['Confirmed'], trace=True, error_action='ignore',
                     start_p = 0, start_q=0, max_p=5, max_q=0,
                     suppress_warnings=True, stepwise=False, seasonal=False)
model_ar.fit(model_train['Confirmed'])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=3004.672, Time=0.02 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=2995.018, Time=0.02 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=2995.651, Time=0.03 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=2990.037, Time=0.06 sec
 ARIMA(4,2,0)(0,0,0)[0] intercept   : AIC=2985.449, Time=0.07 sec
 ARIMA(5,2,0)(0,0,0)[0] intercept   : AIC=2970.848, Time=0.10 sec
Total fit time: 0.317 seconds


ARIMA(order=(5, 2, 0), scoring_args={}, suppress_warnings=True)

In [417]:
prediction_ar = model_ar.predict(len(valid))
y_pred["AR Model Prediction"] = prediction_ar

In [418]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"],
                                              y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ", np.sqrt(mean_squared_error(y_pred["Confirmed"],
                                                                         y_pred["AR Model Prediction"])))

Root Mean Square Error for AR Model:  199901.21425497934


In [419]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["Confirmed"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["Confirmed"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases AR Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [420]:
AR_model_new_prediction = []
for i in range(1, 100):
    AR_model_new_prediction.append(model_ar.predict(len(valid)+i)[-1])

model_predictions["AR Model Prediction"] = AR_model_new_prediction
model_predictions.head()

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516,10573460.804313,10083394.565270
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159,10761314.342326,10229086.950274
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801,10953361.048453,10375078.187901
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443,11147808.718753,10522468.983487
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085,11345481.962612,10669943.278812


#### MA Model (using AUTO ARIMA)

In [421]:
model_train = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid = datewise.iloc[int(datewise.shape[0]*0.9):]
y_pred = valid.copy()

In [422]:
model_ma = auto_arima(model_train["Confirmed"], trace=True, error_action='ignore',
                     start_p=0, start_q=0, max_p=0, max_q=5,
                     suppress_warnings=True, stepwise=False, seasonal=False)
model_ma.fit(model_train["Confirmed"])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=3004.672, Time=0.01 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=2990.744, Time=0.04 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=2995.515, Time=0.05 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=3002.327, Time=0.13 sec
 ARIMA(0,2,4)(0,0,0)[0] intercept   : AIC=2992.933, Time=0.12 sec
 ARIMA(0,2,5)(0,0,0)[0] intercept   : AIC=2983.116, Time=0.13 sec
Total fit time: 0.498 seconds


ARIMA(order=(0, 2, 5), scoring_args={}, suppress_warnings=True)

In [423]:
prediction_ma = model_ar.predict(len(valid))
y_pred["MA Model Prediction"] = prediction_ma

In [424]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=model_train.index, y=model_train['Confirmed'],
                        mode='lines+markers', name='Train Data for Confirmed cases'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Confirmed'],
                        mode='lines+markers', name='Validation Data for Confirmed cases'))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["MA Model Prediction"],
                         mode='lines+markers', name="Prediction for Confirmed Cases"))

fig.update_layout(title='Confirmed Cases MA Model Prediction',
                 xaxis_title='Date', yaxis_title='Confirmed Cases', 
                 legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

In [425]:
MA_model_new_prediction = []

for i in range(1, 100):
    MA_model_new_prediction.append(model_ma.predict(len(valid)+i)[-1])
    
model_predictions["MA Model Prediction"] = MA_model_new_prediction
model_predictions

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516,10573460.804313,10083394.565270,10180535.666030
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159,10761314.342326,10229086.950274,10335245.803169
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801,10953361.048453,10375078.187901,10491322.825764
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443,11147808.718753,10522468.983487,10648766.733816
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085,11345481.962612,10669943.278812,10807577.527323
...,...,...,...,...,...,...,...
94,2020-10-02,11941638.009679,-5339812100.397134,22998915.129886,47802323.311642,27748386.770556,30697944.885460
95,2020-10-03,11995280.288537,-5589944764.441430,23136278.010528,48532856.405936,27980091.813654,30981142.255467
96,2020-10-04,12048922.567395,-5850037808.690240,23273640.891170,49274231.846084,28212718.689182,31265706.510930
97,2020-10-05,12102564.846254,-6120422335.913735,23411003.771812,50026595.166598,28446267.397587,31551637.651849


#### ARIMA Model (using AUTOARIMA)

In [426]:
model_train = datewise.iloc[:int(datewise.shape[0]*0.9)]
valid = datewise.iloc[int(datewise.shape[0]*0.9):]
y_pred = valid.copy()

In [427]:
model_arima = auto_arima(model_train['Confirmed'], trace=True, error_action='ignore',
                        start_p=1, start_q=1, max_p=3, max_q=3,
                        suppress_warnings=True, stepwise=False, seasonal=False)
model_arima.fit(model_train['Confirmed'])

 ARIMA(0,2,0)(0,0,0)[0] intercept   : AIC=3004.672, Time=0.01 sec
 ARIMA(0,2,1)(0,0,0)[0] intercept   : AIC=2990.744, Time=0.03 sec
 ARIMA(0,2,2)(0,0,0)[0] intercept   : AIC=2995.515, Time=0.05 sec
 ARIMA(0,2,3)(0,0,0)[0] intercept   : AIC=3002.327, Time=0.13 sec
 ARIMA(1,2,0)(0,0,0)[0] intercept   : AIC=2995.018, Time=0.03 sec
 ARIMA(1,2,1)(0,0,0)[0] intercept   : AIC=2992.243, Time=0.08 sec
 ARIMA(1,2,2)(0,0,0)[0] intercept   : AIC=2991.789, Time=0.19 sec
 ARIMA(1,2,3)(0,0,0)[0] intercept   : AIC=2984.691, Time=0.20 sec
 ARIMA(2,2,0)(0,0,0)[0] intercept   : AIC=2995.651, Time=0.03 sec
 ARIMA(2,2,1)(0,0,0)[0] intercept   : AIC=2994.027, Time=0.10 sec
 ARIMA(2,2,2)(0,0,0)[0] intercept   : AIC=2992.855, Time=0.18 sec
 ARIMA(2,2,3)(0,0,0)[0] intercept   : AIC=2986.108, Time=0.42 sec
 ARIMA(3,2,0)(0,0,0)[0] intercept   : AIC=2990.037, Time=0.05 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=2985.026, Time=0.09 sec
 ARIMA(3,2,2)(0,0,0)[0] intercept   : AIC=2987.443, Time=0.27 sec
Total fit 

ARIMA(order=(1, 2, 3), scoring_args={}, suppress_warnings=True)

In [428]:
prediction_arima = model_arima.predict(len(valid))
y_pred["ARIMA Model Prediction"] = prediction_arima

In [429]:
model_scores.append(np.sqrt(mean_squared_error(valid['Confirmed'], prediction_arima)))
print("Root Mean Squares Error for ARIMA Model: ", np.sqrt(mean_squared_error(valid['Confirmed'], prediction_arima)))

Root Mean Squares Error for ARIMA Model:  144600.732100827


In [430]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train['Confirmed'],
                        mode='lines+markers', name='Train Data for Confirmed Cases'))
fig.add_trace(go.Scatter(x=valid.index, y=valid['Confirmed'],
                        mode='lines+markers', name='Validation Data for Confirmed cases'))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["ARIMA Model Prediction"],
                         mode='lines+markers', name="Prediction for Confirmed Cases"))

fig.update_layout(title='Confirmed Cases ARIMA Model Prediction',
                 xaxis_title='Date', yaxis_title='Confirmed Cases', 
                 legend=dict(x=0, y=1, traceorder='normal'))

fig.show()

In [431]:
ARIMA_model_new_prediction = []

for i in range(1, 100):
    ARIMA_model_new_prediction.append(model_ma.predict(len(valid)+i)[-1])
    
model_predictions["ARIMA Model Prediction"] = ARIMA_model_new_prediction
model_predictions.head()

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction,ARIMA Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516,10573460.804313,10083394.565270,10180535.666030,10180535.666030
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159,10761314.342326,10229086.950274,10335245.803169,10335245.803169
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801,10953361.048453,10375078.187901,10491322.825764,10491322.825764
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443,11147808.718753,10522468.983487,10648766.733816,10648766.733816
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085,11345481.962612,10669943.278812,10807577.527323,10807577.527323


#### SARIMA Model (using AUTO ARIMA)

In [432]:
model_sarima = auto_arima(model_train["Confirmed"], trace=True, error_action='ignore',
                          start_p=0, start_q=0, max_p=2, max_q=2, m=7,
                          suppress_warnings=True, stepwise=True, seasonal=True)
model_sarima.fit(model_train["Confirmed"])

Performing stepwise search to minimize aic
 ARIMA(0,2,0)(1,0,1)[7]             : AIC=3001.299, Time=0.13 sec
 ARIMA(0,2,0)(0,0,0)[7]             : AIC=3004.116, Time=0.01 sec
 ARIMA(1,2,0)(1,0,0)[7]             : AIC=2982.047, Time=0.06 sec
 ARIMA(0,2,1)(0,0,1)[7]             : AIC=2980.795, Time=0.06 sec
 ARIMA(0,2,1)(0,0,0)[7]             : AIC=2992.249, Time=0.02 sec
 ARIMA(0,2,1)(1,0,1)[7]             : AIC=2966.844, Time=0.15 sec
 ARIMA(0,2,1)(1,0,0)[7]             : AIC=2973.785, Time=0.07 sec
 ARIMA(0,2,1)(2,0,1)[7]             : AIC=2968.760, Time=0.28 sec
 ARIMA(0,2,1)(1,0,2)[7]             : AIC=2968.754, Time=0.26 sec
 ARIMA(0,2,1)(0,0,2)[7]             : AIC=2978.264, Time=0.11 sec
 ARIMA(0,2,1)(2,0,0)[7]             : AIC=2971.508, Time=0.11 sec
 ARIMA(0,2,1)(2,0,2)[7]             : AIC=2970.714, Time=0.59 sec
 ARIMA(1,2,1)(1,0,1)[7]             : AIC=2967.975, Time=0.17 sec
 ARIMA(0,2,2)(1,0,1)[7]             : AIC=2966.962, Time=0.22 sec
 ARIMA(1,2,0)(1,0,1)[7]          

ARIMA(order=(1, 2, 2), scoring_args={}, seasonal_order=(1, 0, 1, 7),
      suppress_warnings=True, with_intercept=False)

In [433]:
prediction_sarima = model_sarima.predict(len(valid))
y_pred["SARIMA Model Prediction"] = prediction_sarima

In [440]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["SARIMA Model Prediction"])))
print("Root Mean Square Error for SARIMA Model: ", np.sqrt(mean_squared_error(y_pred["Confirmed"], y_pred["SARIMA Model Prediction"])))

Root Mean Square Error for SARIMA Model:  235299.4332788909


In [435]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=model_train.index, y=model_train["Confirmed"],
                        mode='lines+markers', name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["Confirmed"],
                        mode='lines+markers', name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["SARIMA Model Prediction"],
                        mode='lines+markers', name="Train Data for Confirmed Cases"))
fig.update_layout(title="Confirmed Cases SARIMA Model Prediction",
                 xaxis_title="Date", yaxis_title="Confirmed Cases", 
                 legend=dict(x=0, y=1, traceorder="normal"))

fig.show()

In [436]:
SARIMA_model_new_prediction = []

for i in range(1, 100):
    SARIMA_model_new_prediction.append(model_sarima.predict(len(valid)+i)[-1])

model_predictions["SARIMA Model Prediction"] = SARIMA_model_new_prediction
model_predictions.head()

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction,ARIMA Model Prediction,SARIMA Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516,10573460.804313,10083394.565270,10180535.666030,10180535.666030,10004531.896463
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159,10761314.342326,10229086.950274,10335245.803169,10335245.803169,10143150.303288
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801,10953361.048453,10375078.187901,10491322.825764,10491322.825764,10290916.942799
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443,11147808.718753,10522468.983487,10648766.733816,10648766.733816,10434719.315996
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085,11345481.962612,10669943.278812,10807577.527323,10807577.527323,10580176.215603


### Summarization of Forecasts using different Models

#### Case1 : Results Using 5% Validation Set

In [379]:
model_predictions_5 = model_predictions
model_predictions_5

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction,ARIMA Model Prediction,SARIMA Model Prediction
0,2020-06-30,7380259.723910,9798236.241235,10374864.021127,10501498.962123,10375748.841769,10477588.784542,10477588.784542,10348088.315152
1,2020-07-01,7438229.365146,9847282.240619,10531464.653123,10686846.257947,10551971.289053,10654689.168253,10654689.168253,10526870.960028
2,2020-07-02,7496199.006382,9881310.459843,10688065.285120,10876342.106420,10708604.876730,10833453.701900,10833453.701900,10683752.536804
3,2020-07-03,7554168.647618,9898732.751112,10844665.917117,11068175.557553,10884340.293289,11013882.385482,11013882.385482,10866114.663773
4,2020-07-04,7612138.288853,9897865.109540,11001266.549113,11263174.884667,11042416.723668,11195975.218999,11195975.218999,11019154.562087
...,...,...,...,...,...,...,...,...,...
94,2020-10-02,12829406.000087,-737621895.380941,25095323.428811,47000933.432577,30400962.700166,34399024.219947,34399024.219947,29130256.888836
95,2020-10-03,12887375.641323,-768602963.472418,25251924.060807,47714190.065643,30663703.285611,34732554.697562,34732554.697562,29367998.321769
96,2020-10-04,12945345.282559,-800575250.229044,25408524.692804,48437955.924928,30927323.988796,35067749.325112,35067749.325112,29582578.623436
97,2020-10-05,13003314.923795,-833560667.977087,25565125.324801,49172373.205895,31192148.704464,35404608.102598,35404608.102598,29818280.019949


In [380]:
print(list(zip(model_names, model_scores)))

[('Linear Regression', 2598361.511152013), ('Polynomial Regression', 339437.01978744625), ("Holt's Linear", 54321.07756790277), ("Holt's Winter Model", 16373.826202717597), ('Auto Regressive Model (AR)', 57962.25983116877), ('Moving Average Model (MA)', 68691.38400632553), ('ARIMA Model', 70212.49513155753), ('SARIMA Model', 70212.49513155753)]


In [381]:
model_names=["Linear Regression", "Polynomial Regression","Holt's Linear","Holt's Winter Model",
            "Auto Regressive Model (AR)","Moving Average Model (MA)","ARIMA Model","SARIMA Model"]
model_summary=pd.DataFrame(zip(model_names,model_scores),columns=["Model Name","Root Mean Squared Error"]).sort_values(["Root Mean Squared Error"])
model_summary

,Model Name,Root Mean Squared Error
3,Holt's Winter Model,16373.826203
2,Holt's Linear,54321.077568
4,Auto Regressive Model (AR),57962.259831
5,Moving Average Model (MA),68691.384006
6,ARIMA Model,70212.495132
7,SARIMA Model,70212.495132
1,Polynomial Regression,339437.019787
0,Linear Regression,2598361.511152


In [382]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Confirmed"],
                        mode='lines+markers', name="Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Holt's Winter Model Prediction"],
                        mode='lines+markers', name="Holt's Winter Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["SARIMA Model Prediction"],
                        mode='lines+markers', name="SARIMA Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Holt's Linear Model Prediction"],
                        mode='lines+markers', name="Holt's Linear Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["AR Model Prediction"],
                        mode='lines+markers', name="AR Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["MA Model Prediction"],
                        mode='lines+markers', name="MA Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["ARIMA Model Prediction"],
                        mode='lines+markers', name="ARIMA Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Linear Regression Prediction"],
                        mode='lines+markers', name="Linear Regression Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Polynomial Regression Prediction"],
                        mode='lines+markers', name="Polynomial Regression Model Prediction"))

fig.update_layout(title="Compare Prediction by the best model with 5% Validation Set",
                 xaxis_title="Date", yaxis_title="Confirmed Cases", 
                 legend=dict(x=0, y=-1, traceorder="normal"))

fig.show()

#### Case2 : Results Using 10% Validation Set

In [441]:
model_predictions_10 = model_predictions
model_predictions_10

,Dates,Linear Regression Prediction,Polynomial Regression Prediction,Holt's Linear Model Prediction,Holt's Winter Model Prediction,AR Model Prediction,MA Model Prediction,ARIMA Model Prediction,SARIMA Model Prediction
0,2020-06-30,6899263.797007,5553059.654346,10086804.349516,10573460.804313,10083394.565270,10180535.666030,10180535.666030,10004531.896463
1,2020-07-01,6952906.075865,4970776.973441,10224167.230159,10761314.342326,10229086.950274,10335245.803169,10335245.803169,10143150.303288
2,2020-07-02,7006548.354724,4300888.528809,10361530.110801,10953361.048453,10375078.187901,10491322.825764,10491322.825764,10290916.942799
3,2020-07-03,7060190.633582,3535363.573804,10498892.991443,11147808.718753,10522468.983487,10648766.733816,10648766.733816,10434719.315996
4,2020-07-04,7113832.912440,2665639.382201,10636255.872085,11345481.962612,10669943.278812,10807577.527323,10807577.527323,10580176.215603
...,...,...,...,...,...,...,...,...,...
94,2020-10-02,11941638.009679,-5339812100.397134,22998915.129886,47802323.311642,27748386.770556,30697944.885460,30697944.885460,25426543.356055
95,2020-10-03,11995280.288537,-5589944764.441430,23136278.010528,48532856.405936,27980091.813654,30981142.255467,30981142.255467,25613392.797263
96,2020-10-04,12048922.567395,-5850037808.690240,23273640.891170,49274231.846084,28212718.689182,31265706.510930,31265706.510930,25795699.216647
97,2020-10-05,12102564.846254,-6120422335.913735,23411003.771812,50026595.166598,28446267.397587,31551637.651849,31551637.651849,25976342.808190


In [442]:
model_scores

[2665388.3159043086,
 1983232.8167772032,
 182429.61166897498,
 52727.7197348411,
 199901.21425497934,
 144600.732100827,
 235299.4332788909,
 235299.4332788909]

In [443]:
model_names=["Linear Regression", "Polynomial Regression","Holt's Linear","Holt's Winter Model",
            "Auto Regressive Model (AR)","Moving Average Model (MA)","ARIMA Model","SARIMA Model"]
model_summary=pd.DataFrame(zip(model_names,model_scores),columns=["Model Name","Root Mean Squared Error"]).sort_values(["Root Mean Squared Error"])
model_summary

,Model Name,Root Mean Squared Error
3,Holt's Winter Model,52727.719735
5,Moving Average Model (MA),144600.732101
2,Holt's Linear,182429.611669
4,Auto Regressive Model (AR),199901.214255
6,ARIMA Model,235299.433279
7,SARIMA Model,235299.433279
1,Polynomial Regression,1983232.816777
0,Linear Regression,2665388.315904


In [444]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Confirmed"],
                        mode='lines+markers', name="Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Holt's Winter Model Prediction"],
                        mode='lines+markers', name="Holt's Winter Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["SARIMA Model Prediction"],
                        mode='lines+markers', name="SARIMA Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Holt's Linear Model Prediction"],
                        mode='lines+markers', name="Holt's Linear Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["AR Model Prediction"],
                        mode='lines+markers', name="AR Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["MA Model Prediction"],
                        mode='lines+markers', name="MA Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["ARIMA Model Prediction"],
                        mode='lines+markers', name="ARIMA Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Linear Regression Prediction"],
                        mode='lines+markers', name="Linear Regression Model Prediction"))
fig.add_trace(go.Scatter(x=model_predictions["Dates"], y=model_predictions["Polynomial Regression Prediction"],
                        mode='lines+markers', name="Polynomial Regression Model Prediction"))

fig.update_layout(title="Compare Prediction by the best model with 10% Validation Set",
                 xaxis_title="Date", yaxis_title="Confirmed Cases", 
                 legend=dict(x=0, y=-1, traceorder="normal"))

fig.show()

#### The Best Model is "Holt's Winter Model"

In [447]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=datewise.index, y=datewise["Confirmed"],
                        mode='markers', name="Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=model_predictions_5["Dates"], y=model_predictions_5["Holt's Winter Model Prediction"],
                        mode='markers', name="Holt's Winter Model Prediction with 5% Valid Set"))
fig.add_trace(go.Scatter(x=model_predictions_10["Dates"], y=model_predictions_10["Holt's Winter Model Prediction"],
                        mode='markers', name="Holt's Winter Model Prediction with 10% Valid Set"))

fig.update_layout(title="Compare The Best Model by Validation Set",
                 xaxis_title="Date", yaxis_title="Confirmed Cases", 
                 legend=dict(x=0, y=1, traceorder="normal"))

fig.show()

In [457]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=datewise.index, y=(datewise["Confirmed"]-datewise["Confirmed"].shift().fillna(0)),
                        mode='markers', name="Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=model_predictions_5["Dates"], y=(model_predictions_5["Holt's Winter Model Prediction"]-model_predictions_5["Holt's Winter Model Prediction"].shift()),
                        mode='markers', name="Holt's Winter Model Prediction with 5% Valid Set"))
fig.add_trace(go.Scatter(x=model_predictions_10["Dates"], y=(model_predictions_10["Holt's Winter Model Prediction"]-model_predictions_10["Holt's Winter Model Prediction"].shift()),
                        mode='markers', name="Holt's Winter Model Prediction with 10% Valid Set"))

fig.update_layout(title="Amount of Change by a day",
                 xaxis_title="Date", yaxis_title="Confirmed Cases", 
                 legend=dict(x=0, y=1, traceorder="normal"))

fig.show()

In [448]:
datewise

,ObservationDate,Confirmed,Recovered,Deaths,Days Since,WeekOfYear,Mortality Rate,Recovery Rate,Active Cases,Closed Cases
ObservationDate,,,,,,,,,,
2020-01-22,2020-01-22,555.000000,28.000000,17.000000,0,4,3.063063,5.045045,510.000000,45.000000
2020-01-23,2020-01-23,653.000000,30.000000,18.000000,1,4,2.756508,4.594181,605.000000,48.000000
2020-01-24,2020-01-24,941.000000,36.000000,26.000000,2,4,2.763018,3.825717,879.000000,62.000000
2020-01-25,2020-01-25,1438.000000,39.000000,42.000000,3,4,2.920723,2.712100,1357.000000,81.000000
2020-01-26,2020-01-26,2118.000000,52.000000,56.000000,4,4,2.644004,2.455146,2010.000000,108.000000
...,...,...,...,...,...,...,...,...,...,...
2020-06-25,2020-06-25,9609829.000000,4838921.000000,489312.000000,155,26,5.091787,50.353872,4281596.000000,5328233.000000
2020-06-26,2020-06-26,9801572.000000,4945557.000000,494181.000000,156,26,5.041855,50.456774,4361834.000000,5439738.000000
2020-06-27,2020-06-27,9979535.000000,5051864.000000,498710.000000,157,26,4.997327,50.622238,4428961.000000,5550574.000000


In [449]:
datewise.shift()


,ObservationDate,Confirmed,Recovered,Deaths,Days Since,WeekOfYear,Mortality Rate,Recovery Rate,Active Cases,Closed Cases
ObservationDate,,,,,,,,,,
2020-01-22,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan
2020-01-23,2020-01-22,555.000000,28.000000,17.000000,0.000000,4.000000,3.063063,5.045045,510.000000,45.000000
2020-01-24,2020-01-23,653.000000,30.000000,18.000000,1.000000,4.000000,2.756508,4.594181,605.000000,48.000000
2020-01-25,2020-01-24,941.000000,36.000000,26.000000,2.000000,4.000000,2.763018,3.825717,879.000000,62.000000
2020-01-26,2020-01-25,1438.000000,39.000000,42.000000,3.000000,4.000000,2.920723,2.712100,1357.000000,81.000000
...,...,...,...,...,...,...,...,...,...,...
2020-06-25,2020-06-24,9430516.000000,4746118.000000,482753.000000,154.000000,26.000000,5.119052,50.327236,4201645.000000,5228871.000000
2020-06-26,2020-06-25,9609829.000000,4838921.000000,489312.000000,155.000000,26.000000,5.091787,50.353872,4281596.000000,5328233.000000
2020-06-27,2020-06-26,9801572.000000,4945557.000000,494181.000000,156.000000,26.000000,5.041855,50.456774,4361834.000000,5439738.000000
